In [ ]:
# import torch
# print(f"\n🔍 PyTorch: {torch.__version__}")
# print(f"🔍 GPU: {torch.cuda.is_available()}")
# print(f"🔍 Nome da GPU: {torch.cuda.get_device_name(0)}")

# # Nova forma de verificar ROCm a partir do PyTorch 2.0+
# print(f"🔍 Backend GPU: {'ROCm' if torch.version.hip else 'CUDA'}")
# print(f"🔍 Versão HIP: {torch.version.hip}\n")

# Carregando Pacotes e Tokenizador:

In [1]:
from transformers import pipeline
import torch
import pdfplumber

from transformers import AutoTokenizer


In [2]:
# Verifica e configura o dispositivo (GPU/CPU)
device = 0 if torch.cuda.is_available() else -1  # 0 = GPU, -1 = CPU
print(f"🔧 Rodando em: {'GPU' if device == 0 else 'CPU'}")

🔧 Rodando em: GPU


In [ ]:
# # Carregar modelo BERTimbau pré-treinado para Q&A em Ingles
# tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

# qa_pipeline = pipeline(
#     "question-answering",
#     model="bert-large-uncased-whole-word-masking-finetuned-squad",  # Modelo em inglês
#     tokenizer="bert-large-uncased-whole-word-masking-finetuned-squad",
#     device=device
# )

In [16]:
# Carregar modelo BERTimbau pré-treinado para Q&A em Portugues
tokenizer = AutoTokenizer.from_pretrained("pierreguillou/bert-large-cased-squad-v1.1-portuguese")

qa_pipeline = pipeline(
    "question-answering",
    model="pierreguillou/bert-large-cased-squad-v1.1-portuguese",  # Modelo em Portugues
    tokenizer="pierreguillou/bert-large-cased-squad-v1.1-portuguese",
    device=device
)

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/918 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/210k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Device set to use cuda:0


# Tratamento do Documento

In [17]:
def carregar_documento(caminho_do_pdf):

    with pdfplumber.open(caminho_do_pdf) as pdf:
        texto = " ".join(
            page.extract_text() for page in pdf.pages 
            if page.extract_text()  # Ignora páginas sem texto
        )
        texto = " ".join(texto.split())  # Normaliza espaços
    return texto

In [18]:
def dividir_em_chunks_tokenizados(texto):

    chunk_size = 400
    overlap = 100

    tokens = tokenizer.tokenize(texto)
    chunks = []
    for i in range(0, len(tokens), chunk_size - overlap):
        chunk = tokens[i:i + chunk_size]
        chunks.append(tokenizer.convert_tokens_to_string(chunk))
    return chunks

# Definição de Processamento das Perguntas e das Respostas

In [19]:
def processar_pergunta(pergunta, documento):
    chunks = dividir_em_chunks_tokenizados(documento)
    
    # 1. Tokeniza a pergunta para verificar tamanho
    tokens_pergunta = tokenizer.tokenize(pergunta)
    if len(tokens_pergunta) > 60:  # Limite arbitrário (ajuste conforme necessário)
        print("Pergunta muito longa! Simplifique para melhor precisão.")
    
    # 2. Executa Q&A em cada chunk
    respostas = []
    for chunk in chunks:
        try:
            resposta = qa_pipeline(question=pergunta, context=chunk)
            respostas.append(resposta)

            resposta_completa = {
                'answer': resposta.get('answer', ''),
                'score': resposta.get('score', 0),
                'context': resposta.get('context', chunk[:500])  # Fallback: 500 primeiros chars
            }
            respostas.append(resposta_completa)
        except Exception as e:
            print(f"Erro no chunk: {str(e)}")
            continue
    
    # 3. Filtra respostas com score baixo e seleciona a melhor
    respostas_validas = [r for r in respostas if r['score'] >= 0.1]
    if not respostas_validas:
        print("Não há resposta sobre isto nesse documento.")
        return None
    
    return max(respostas_validas, key=lambda x: x['score'])

# Pergunta e Resposta

In [43]:
#Carrega o documento em PDF

documento = carregar_documento("Cabelo Cacheado Maior.pdf")

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


In [54]:
#Pergunta sobre o documento
pergunta = "Qual o título do texto?"

In [55]:
#Resposta gerada
resposta = processar_pergunta(pergunta, documento)
print(resposta)

Não há resposta sobre isto nesse documento.
None
